In [1]:
# wav퍼앍허 transcript.csv 디렉토리에 저장
import fnmatch
import os
import zipfile
import tarfile
import os
import shutil
import fnmatch

def move_files(src_dir, out_dir):
    audio_dir = os.path.join(out_dir, 'audio')
    transcripts_dir = os.path.join(out_dir, 'transcripts')
    os.makedirs(audio_dir, exist_ok=True)
    os.makedirs(transcripts_dir, exist_ok=True)

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.wav'):
                shutil.move(os.path.join(root, file), audio_dir)
            elif fnmatch.fnmatch(file.lower(), '*transcript.csv'):
                shutil.move(os.path.join(root, file), transcripts_dir)

if __name__ == '__main__':
    # directory containing unzipped DAIC-WOZ files
    src_dir = 'D:/daic-woz!'

    # directory where audio and transcripts folders will be created
    out_path = 'C:/pknu_6/우울증진단/output'

    # move .wav and *transcript.csv files in src_dir to appropriate folders
    move_files(src_dir, out_path)


Error: Destination path 'C:/pknu_6/우울증진단/output\audio\300_AUDIO.wav' already exists

### 침묵제거, ellie 목소리 제거

In [2]:
import os
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS
import scipy.io.wavfile as wavfile
import numpy as np


def remove_silence(filename, out_dir, smoothing=1.0, weight=0.3, store_segments=False, plot=False):
    participant_id = 'P' + os.path.basename(filename).split('_')[0]  # PXXX
    if can_segment(participant_id):
        # 참가자별 음성 파일이 저장될 디렉토리 생성
        participant_dir = os.path.join(out_dir, participant_id)
        if not os.path.exists(participant_dir):
            os.makedirs(participant_dir)

        [fs, x] = aIO.read_audio_file(filename)
        segments = aS.silence_removal(x, fs, 0.020, 0.020, smooth_window=smoothing, weight=weight, plot=plot)

        out_file = '{}_no_silence.wav'.format(participant_id)
        x_no_silence = np.array([], dtype='int16')
        for s in segments:
            segment_no_silence = x[int(fs * s[0]):int(fs * s[1])]
            if store_segments:  # 묵음이 제거된 음성 세그먼트를 참가자별 폴더 내에 WAV 파일로 저장
                segment_name = "{:s}_{:.2f}-{:.2f}.wav".format(participant_id, s[0], s[1])
                wavfile.write(os.path.join(participant_dir, segment_name), fs, segment_no_silence)
            x_no_silence = np.append(x_no_silence, segment_no_silence)

        # 묵음이 제거된 음성 파일을 참가자별 폴더 내에 저장
        wavfile.write(os.path.join(participant_dir, out_file), fs, x_no_silence)


def can_segment(participant_id):
    troubled = {}
    return participant_id not in troubled


if __name__ == '__main__':
    # 원시 WAV 파일이 있는 디렉토리
    dir_name = 'C:/pknu_6/우울증진단/output/audio'

    # 참가자별 폴더가 생성될 디렉토리 및 가공된 음성 파일이 저장될 디렉토리
    out_path = 'C:/pknu_6/우울증진단/output/interim'

    # dir_name의 WAV 파일들을 반복하며 가공된 음성 파일 생성
    for root, dirs, files in os.walk(dir_name):
        for file in files:
            if file.endswith('.wav'):
                file_name = os.path.join(root, file)
                remove_silence(file_name, out_path)

C:\Users\Administrator\anaconda3\envs\pknu_SF5\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 20752044 bytes, expected 166016044 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
C:\Users\Administrator\anaconda3\envs\pknu_SF5\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 26364844 bytes, expected 210918444 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io
C:\Users\Administrator\anaconda3\envs\pknu_SF5\lib\site-packages\pyAudioAnalysis\audioBasicIO.py:99: WavFileWarning: Reached EOF prematurely; finished at 24281644 bytes, expected 194252844 bytes from header.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


KeyboardInterrupt: 